# Exemplo de GridSearchCV do [blog](gusrabbit.com) do Gustavo

Primeiro importamos o que vamos usar:

In [34]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Vamos carregar os dados do dataset de boston do scikit. Esse dataset tem características e preços de casas em boston, vamos usar para fazer uma regressão! Se você apertar **tab** depois do ponto você vai conseguir ver outros datasets que já vem no scikit.

In [3]:
housing = datasets.load_boston()

Que nem da [outra vez](https://gusrabbit.com/code/cross_validate/), a gente separa a variável objetivo das features:

In [4]:
X = housing.data
y = housing.target

Vamos pegar um modelo basicão, o DummyRegressor. Ele é um regressor cuja previsão é aleatória. É legal usar esses modelos aleatórios como baseline para comparar a qualidade dos nossos modelos. No mínino tem que ser melhor que isso. Vamos instanciar ele:

In [6]:
baseline = DummyRegressor()

De acordo com a [documentação](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) ele precisa dos parâmetros a seguir:

GridSearchCV(estimator, **param_grid**, **scoring=None**, fit_params=None, n_jobs=1, iid=True, refit=True, cv=None, verbose=0, pre_dispatch=‘2\*n_jobs’, error_score=’raise’, return_train_score=’warn’)

Mas primeiro! Vamos definir as métricas que queremos medir, lembrando que todas estão no [link do amor](http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter). Vamos definir uma lista com todas as de regressão:

In [7]:
metricas = ['explained_variance', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'r2']

Lembrando que o tchan do Grid Search é exatamente ele fazer os cross validation de vários modelos com hiperparâmetros diferentes de uma vez. Então vamos definir o **param_grid** que vai definir quais modelos com quais parâmetros o nosso Grid Search vai rodar.

Se você olhar na [documentação](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) ela diz que pode ser um dicionário ou uma lista de dicionários, qual a ideia?

A vibe é definir um dicionário com os parâmetros do nosso estimador. Vamos usar o método get_params pra ver o nome certinho dos parâmetros do nosso DummyRegressor:

In [10]:
baseline.get_params().keys()

dict_keys(['constant', 'quantile', 'strategy'])

Essas são as chaves do nosso dict, os valores são aqueles que queremos que o Grid Search rode. Criamos um dict assim:

In [19]:
hyper = {'strategy':['mean', 'median', 'quantile', 'constant'],
         'quantile':[.75],
         'constant':[300000]}

Dessa forma, o nosso Grid Search vai rodar quatro modelos, cada um com uma das estratégias. Isso acontece, pois os parâmetros quantile e constant só são usados naquelas duas estratégias. Vamos definir o Grid Search com tudo que a gente montou até agora e com *verbose=100* pra ele ir falando o que ele ta fazendo passo a passo. Vamos mandar ele fazer o refit no *neg_mean_squared_error*, mas só porque esse é a minha métricas preferida. Isso significa que ele vai escolher o modelo com o menor erro quadrado médio. E por último, a gente pede pra n retornar os scores de treinamento porque eles são meio inúteis.

In [20]:
meu_primeiro_grid = GridSearchCV(baseline, param_grid=hyper, scoring=metricas, verbose=100, refit='neg_mean_squared_error', return_train_score=False)

Por padrão, essa função vai separar o dataset em 3 partes e vai devolver a métrica média de cada uma dessas 3. No futuro eu vou postar sobre Kfold e como podemos separar os dados em várias partes para fazer o cross validation como aqui. Tudo isso é controlado pelo parâmetro *cv*. Agora vamos mandar o Grid Search rodar nos dados de boston!

In [21]:
meu_primeiro_grid.fit(X,y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] constant=300000, quantile=0.75, strategy=mean ...................
[CV]  constant=300000, quantile=0.75, strategy=mean, explained_variance=0.0, neg_mean_absolute_error=-5.0932488192018, neg_mean_squared_error=-50.81033661000395, neg_mean_squared_log_error=-0.07863554756074104, neg_median_absolute_error=-3.697626112759643, r2=-0.024692018774538305, total=   0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[CV] constant=300000, quantile=0.75, strategy=mean ...................
[CV]  constant=300000, quantile=0.75, strategy=mean, explained_variance=0.0, neg_mean_absolute_error=-9.21102312433059, neg_mean_squared_error=-154.44755358848917, neg_mean_squared_log_error=-0.1786768930506994, neg_median_absolute_error=-5.467359050445104, r2=-1.0939806689130491, total=   0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[CV] constant=300000, quantile=0.75, strategy

GridSearchCV(cv=None, error_score='raise',
       estimator=DummyRegressor(constant=None, quantile=None, strategy='mean'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'strategy': ['mean', 'median', 'quantile', 'constant'], 'quantile': [0.75], 'constant': [300000]},
       pre_dispatch='2*n_jobs', refit='neg_mean_squared_error',
       return_train_score=False,
       scoring=['explained_variance', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'r2'],
       verbose=100)

Agora vamos dar uma olhada nos resultados, o melhor estimador foi:

In [24]:
meu_primeiro_grid.best_estimator_

DummyRegressor(constant=300000, quantile=0.75, strategy='median')

E o melhor score foi:

In [27]:
meu_primeiro_grid.best_score_

-103.223814229249

Que diabéisso? Ok, como a gente colocou *neg_mean_squared_error* no refite ele está devolvendo o melhor score dessa métrica. Essa métrica da a média do quadrado dos erros, só q negativa. Por que negativa? Acho que é pra ficar mais fácil de minimizar, aí eles adotam o padrão de colocar as métricas tudo negativa. O erro é a diferença entre o que o modelo preveu e o valor verdadeiro da casa. Aqui como a melhor estratégia foi a mediana, a previsão do modelo foi a mediana dos valores de treinamento pra toda casa nova. A gente pega a diferença do prebisto pelo verdadeiro, eleva ao quadrado (normal em estatística pra evitar que erros negativos e positivos se anulem, e é mais fácil de derivar do que usar módeulo).

Então pra ter uma ideia se isso ta bom ou ruim basta tirar a raiz disso:

In [31]:
np.sqrt(meu_primeiro_grid.best_score_*-1)

10.159912117201063

Em média nosso modelo errou o preço por 10 mil dólares. O Grid Search retorna um dicionário com todos os resultados, eu gosto de transformar ele num dataframe pra ficar mais fácil de ver (antes vou usar uma manhã do pandas pra que ele mostre todas as colunas):

In [36]:
pd.set_option('max_columns',200)
pd.DataFrame(meu_primeiro_grid.cv_results_)

,mean_fit_time,mean_score_time,mean_test_explained_variance,mean_test_neg_mean_absolute_error,mean_test_neg_mean_squared_error,mean_test_neg_mean_squared_log_error,mean_test_neg_median_absolute_error,mean_test_r2,param_constant,param_quantile,param_strategy,params,rank_test_explained_variance,rank_test_neg_mean_absolute_error,rank_test_neg_mean_squared_error,rank_test_neg_mean_squared_log_error,rank_test_neg_median_absolute_error,rank_test_r2,split0_test_explained_variance,split0_test_neg_mean_absolute_error,split0_test_neg_mean_squared_error,split0_test_neg_mean_squared_log_error,split0_test_neg_median_absolute_error,split0_test_r2,split1_test_explained_variance,split1_test_neg_mean_absolute_error,split1_test_neg_mean_squared_error,split1_test_neg_mean_squared_log_error,split1_test_neg_median_absolute_error,split1_test_r2,split2_test_explained_variance,split2_test_neg_mean_absolute_error,split2_test_neg_mean_squared_error,split2_test_neg_mean_squared_log_error,split2_test_neg_median_absolute_error,split2_test_r2,std_fit_time,std_score_time,std_test_explained_variance,std_test_neg_mean_absolute_error,std_test_neg_mean_squared_error,std_test_neg_mean_squared_log_error,std_test_neg_median_absolute_error,std_test_r2
0,0.000671,0.002539,0.000000e+00,-8.018024,-1.111172e+02,-0.206390,-6.366456,-6.885605e-01,300000,0.75,mean,"{'constant': 300000, 'quantile': 0.75, 'strate...",1,2,2,2,2,2,0.000000e+00,-5.093249,-5.081034e+01,-0.078636,-3.697626,-2.469202e-02,0.000000e+00,-9.211023,-1.544476e+02,-0.178677,-5.467359,-1.093981e+00,0.000000e+00,-9.760108,-1.281946e+02,-0.362782,-9.955621,-9.485471e-01,0.000227,0.001006,0.000000e+00,2.083278,4.402961e+01,0.117594,2.631723,4.738525e-01
1,0.000743,0.001875,0.000000e+00,-7.449605,-1.032238e+02,-0.185305,-5.578854,-5.621859e-01,300000,0.75,median,"{'constant': 300000, 'quantile': 0.75, 'strate...",1,1,1,1,1,1,0.000000e+00,-4.763905,-4.958598e+01,-0.072742,-3.000000,-3.459903e-07,0.000000e+00,-9.583432,-1.624073e+02,-0.192958,-5.900000,-1.201898e+00,0.000000e+00,-8.004762,-9.764512e+01,-0.290841,-7.850000,-4.841974e-01,0.000097,0.000262,0.000000e+00,2.008020,4.627207e+01,0.089159,1.992041,4.942254e-01
2,0.001881,0.002124,0.000000e+00,-9.471443,-1.319885e+02,-0.250808,-8.631621,-1.091292e+00,300000,0.75,quantile,"{'constant': 300000, 'quantile': 0.75, 'strate...",1,3,3,3,3,3,0.000000e+00,-8.267456,-8.684651e+01,-0.147346,-8.000000,-7.514335e-01,0.000000e+00,-7.147929,-1.075766e+02,-0.106111,-4.200000,-4.585097e-01,0.000000e+00,-13.019940,-2.019565e+02,-0.500445,-13.725000,-2.069722e+00,0.001280,0.001010,0.000000e+00,2.543220,5.005038e+01,0.176801,3.912279,7.001137e-01
3,0.000403,0.001779,-1.462699e-13,-299977.467194,-8.998648e+10,-90.896113,-299979.408300,-1.467723e+09,300000,0.75,constant,"{'constant': 300000, 'quantile': 0.75, 'strate...",4,4,4,4,4,4,-3.226308e-13,-299978.204142,-8.998692e+10,-90.769786,-299979.600000,-1.814766e+09,-2.020606e-13,-299971.484615,-8.998289e+10,-85.911314,-299975.000000,-1.219977e+09,8.726353e-14,-299982.744048,-8.998965e+10,-96.037662,-299983.650000,-1.367835e+09,0.000248,0.000239,1.718582e-13,4.623897,2.774121e+06,4.132971,3.532197,2.530577e+08


Vamos ver se uma regressão linear simples fica melhor? Bora instanciar!

In [37]:
ols = LinearRegression()

In [39]:
ols.get_params().keys()

dict_keys(['copy_X', 'fit_intercept', 'n_jobs', 'normalize'])

Os únicos parâmetros interessantes para a gente são o *fit_intercept* e o *normalize*:

In [40]:
ols_params = {'fit_intercept':[True, False],
              'normalize':[True, False]}

Vamos definir o Grid Search!

In [42]:
meu_segundo_grid = GridSearchCV(ols, param_grid=ols_params, scoring=metricas, verbose=100, refit='neg_mean_squared_error', return_train_score=False)

In [43]:
meu_segundo_grid.fit(X, y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] fit_intercept=True, normalize=True ..............................
[CV]  fit_intercept=True, normalize=True, explained_variance=0.5970136165050308, neg_mean_absolute_error=-3.3288087161058484, neg_mean_squared_error=-20.68720738696552, neg_mean_squared_log_error=-0.08111839971788834, neg_median_absolute_error=-2.3841241723561346, r2=0.58280110476608, total=   0.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[CV] fit_intercept=True, normalize=True ..............................
[CV]  fit_intercept=True, normalize=True, explained_variance=0.541285783038106, neg_mean_absolute_error=-4.282203356265531, neg_mean_squared_error=-34.52324169521958, neg_mean_squared_log_error=-0.034322552052381425, neg_median_absolute_error=-3.152069451528778, r2=0.531938194821685, total=   0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[CV] fit_intercept=True, normalize=True

GridSearchCV(cv=None, error_score='raise',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'fit_intercept': [True, False], 'normalize': [True, False]},
       pre_dispatch='2*n_jobs', refit='neg_mean_squared_error',
       return_train_score=False,
       scoring=['explained_variance', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'r2'],
       verbose=100)

In [44]:
meu_segundo_grid.best_estimator_

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [45]:
meu_segundo_grid.best_score_

-168.08917760165718

In [46]:
np.sqrt(meu_segundo_grid.best_score_*-1)

12.964921041088418

A regressão ta dando um erro de 12!!! Pior do que sempre chutar a mediana! 